In [1]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [2]:
import deep_learning

2024-05-24 11:31:47.935629: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-24 11:31:48.000006: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 11:31:48.693450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
with open("full_data.pic", "rb") as r:
    seeds_total, traces_total, key, round_perms_labels, copy_perms_labels, masks_labels, rws_perms_labels, rws_masks_labels = pic.load(r)

In [4]:
try:
    deep_learning.check_file_exists("./resnet_models/resnet_rws_300000.keras")
    from tensorflow.keras.models import load_model
    resnet = load_model("./resnet_models/resnet_rws_300000.keras")
except ValueError:
    resnet = deep_learning.ResNetSCA(rws=True)
    

2024-05-24 11:32:23.071918: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:41:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-24 11:32:23.072406: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-24 11:32:23.207552: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:41:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-24 11:32:23.207610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-24 11:32:23.207637: I external/local_xla/xla/stream_executor

In [5]:
X_total, y_total = resnet.prepare_data_dl(traces_total, round_perms_labels, copy_perms_labels, masks_labels, rws_perms_labels, rws_masks_labels)

X_profiling, X_extraction = train_test_split(X_total, test_size=100_000, random_state=0)
y_profiling = {}
y_extraction = {}
for label in y_total.keys():
    train, test = train_test_split(y_total[label], test_size=100_000, random_state=0)
    y_profiling[label] = train
    y_extraction[label] = test

In [ ]:
history = resnet.train_model(X_profiling, y_profiling, "./resnet_models/resnet_rws_300000.keras", epochs=1000, batch_size=16, patience=50, validation_split=0.1)

In [48]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical

y_predicted = load_model("./resnet_models/resnet_350000_52_epochs.keras").predict(X_extraction)
for i, k in enumerate(y_extraction.keys()):
    print(f"{k}: {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k])}/{y_extraction[k].shape[0]} = {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k]) / y_extraction[k].shape[0]}")

np.mean([categorical_crossentropy(to_categorical(y_extraction[k]), y_predicted[i]) for i, k in enumerate(y_extraction.keys())], axis=1)

1411/1411 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step
x_round: 24419/45133 = 0.5410453548401392
block_perm_0: 39134/45133 = 0.8670817362018922
mask_0_0: 354/45133 = 0.007843484811556953
mask_0_1: 365/45133 = 0.00808720891587087
mask_0_2: 305/45133 = 0.006757804710522234
mask_0_3: 302/45133 = 0.006691334500254802
mask_0_4: 332/45133 = 0.0073560366029291205
mask_0_5: 305/45133 = 0.006757804710522234
mask_0_6: 276/45133 = 0.006115259344603727


array([1.3536543, 0.3749589, 5.503599 , 5.5072737, 5.477    , 5.5458245,
       5.490437 , 5.568437 , 5.656007 ], dtype=float32)